In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
!pip install -q keras

In [0]:
!pip install -q pydrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding,Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

In [0]:

from keras import callbacks
from keras.models import Sequential
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense,Dropout,Embedding
from keras.optimizers import Adam,SGD


def create_BiLSTM_model(vocabulary_size,embedding_size,embedding_matrix):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], trainable=False))
    model_glove.add(Bidirectional(LSTM(100)))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_glove.summary()
    return model_glove


def callback(checkpoint_path,tf_log_dir_name='./tf-log/',patience_lr=10,):
    cb = []
    """
    Tensorboard log callback
    """
    tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    cb.append(tb)

    """
    Model-Checkpoint
    """
    m = callbacks.ModelCheckpoint(filepath=checkpoint_path,monitor='val_loss',mode='auto',save_best_only=True)
    cb.append(m)

    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    cb.append(reduce_lr_loss)

    """
    Early Stopping callback
    """
    # Uncomment for usage
    early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
    cb.append(early_stop)



    return cb

######### Show Train Val History Graph ###############
def plot_loss_accu(history,lossLoc='Train_Val_Loss',accLoc='Train_Val_acc'):
    import matplotlib.pyplot as plt

    plt.clf()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    #plt.show()
    plt.savefig(lossLoc)

    plt.clf()

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    #plt.show()
    plt.savefig(accLoc)


In [0]:
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from sklearn.model_selection import KFold
import gc
import numpy as np
import keras.backend as K


In [0]:
if len(K.tensorflow_backend._get_available_gpus()) > 0:
  print("GPU Available !!!!!")
  from keras.layers import CuDNNLSTM as LSTM
  from keras.layers import CuDNNGRU as GRU

In [0]:

pickle_load = open('Reuter_C50/pickle_train_Reuter_C50_All_train_1_100dim.pickle', 'rb')
X_train,y_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_test_Reuter_C50_All_test_1_100dim.pickle', 'rb')
X_test,y_test,_=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_cleanXy_Reuter_C50_All_train_1.pickle', 'rb')
x1_train,y1_train=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_cleanXy_Reuter_C50_All_test_1.pickle', 'rb')
x1_test,y1_test=pickle.load(pickle_load)


vocabulary_size = embedding_matrix.shape[0]
embedding_size=100


kfold = KFold(n_splits=5, shuffle=True, random_state=42)##################
cvscores=[]
classfication_report=[]

In [0]:
# import for showing the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# **Model Architecture**

In [0]:
import keras
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense,Dropout,Embedding

max_words_len=300
#def create_Att_Model(max_words_len, embedding_size, embedding_matrix):
encoder_input=keras.Input(shape=(max_words_len,),)
encoder_embedding = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_size,
                              weights=[embedding_matrix],
                              input_length=max_words_len)


encoder_embedding_layer=encoder_embedding(encoder_input)
#layer1=Bidirectional(LSTM(NN_dim_1,return_sequences=True))
#encoder_output=layer1(encoder_embedding_layer)

conv1D_1=Conv1D(512, 4, activation='relu',name='con1')(encoder_embedding_layer)
maxPool1D_1=MaxPooling1D(pool_size=2,name='maxpool1')(conv1D_1)

conv1D_2=Conv1D(256, 3, activation='relu',name='con2')(maxPool1D_1)
maxPool1D_2=MaxPooling1D(pool_size=2,name='maxpool2')(conv1D_2)

conv1D_3=Conv1D(128, 2, activation='relu',name='con3')(maxPool1D_2)
maxPool1D_3=MaxPooling1D(pool_size=2,name='maxpool3')(conv1D_3)




# Apply Bidirectional LSTM over embedded inputs
lstm_outs = keras.layers.wrappers.Bidirectional(
    keras.layers.LSTM(100, return_sequences=True)
)(maxPool1D_3)



# Attention Mechanism - Generate attention vectors
input_dim = int(lstm_outs.shape[2])
permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
print(attention_vector.shape)
attention_vector = keras.layers.Reshape((attention_vector.shape[1],))(attention_vector)
attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])

# Last layer: fully connected with softmax activation
fc = keras.layers.Dense(100, activation='relu')(attention_output)
output = keras.layers.Dense(50, activation='softmax')(fc)

# Finally building model
model = keras.Model(inputs=[encoder_input], outputs=output)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

# Print model summary
model.summary()



In [0]:
import os
import tensorflow as tf
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = callbacks.ModelCheckpoint(filepath=checkpoint_path,monitor='val_loss',mode='auto',save_best_only=True)

# **Reuter C50 All**

In [0]:

history=model.fit(X_train, y_train, validation_split=0.1, epochs=50, batch_size=128,shuffle=True)


In [0]:
# Re-create the model to get attention vectors as well as label prediction
model_with_attentions = keras.Model(inputs=model.input,
                                    outputs=[model.output, 
                                             model.get_layer('attention_vec').output])

In [0]:
from sklearn.metrics import precision_recall_fscore_support, classification_report,accuracy_score

y_pred, attentions = model_with_attentions.predict(X_test)

for y in y_pred:
  #print(y)
  for i in range(len(y)):
    #pass
    y[i]=int(y[i]>0.5)

In [0]:
import pickle
with open('Predictions/pickle_Pred_Reuter_C50_All_1.pickle','wb') as f:
    pickle.dump((y_test,y_pred),f)

In [0]:
from sklearn.metrics import precision_recall_fscore_support, classification_report,accuracy_score

print('Accuracy Report:')
print(accuracy_score(y_test,y_pred))


In [0]:
class_names = ["0","1"]

# Generate the confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Spooky Author Classification Confusion matrix')
'''plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Spooky Author Classification Confusion matrix, without normalization')'''

plt.show()

# **Freezing Layers**

In [0]:
weightsAndBias = model.get_layer(name='con1').get_weights()
weights_before = model.get_layer(name='con1').get_weights()[0]
# freeze the weights of this layer
model.get_layer(name='con1').trainable = False
model.get_layer(name='maxpool1').trainable = False
model.get_layer(name='con2').trainable = False
model.get_layer(name='maxpool2').trainable = False
model.get_layer(name='con3').trainable = False
model.get_layer(name='maxpool3').trainable = False

# **OUTPUT LAYER CHANGED**

In [0]:
output = keras.layers.Dense(1, activation='sigmoid')(fc)
#output = keras.layers.Dense(3, activation='softmax')(fc)
#output = keras.layers.Dense(50, activation='softmax')(fc)

In [0]:
# Finally building model
model = keras.Model(inputs=[encoder_input], outputs=output)
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer='adam')
#model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

# Print model summary
model.summary()


# **Quran Bukhari Authorship Classification**

In [0]:
print("Fitting Model...")

pickle_load = open('Quran_Bukhari_Classification/pickle_train_Quran_Bukhari_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Quran_Bukhari_Classification/pickle_test_Quran_Bukhari_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('pickle_cleanXy_Quran_Bukhari_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-2360,:],y1_random[:-2360,:],x1_random[-2360:,:],y1_random[-2360:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)


# evaluate the model
print("Evaluating Model...")
##########################################
y1_pred, attentions = model_with_attentions.predict(X1_test)
y_pred_class=[]

for y in y1_pred:
  y[0]=int(y[0]>0.5)

from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score

acc=accuracy_score(y1_test,y1_pred)

print("Eval with Quran Bukhari: "+str(Fold)+" " +str(acc))


In [0]:
import pickle
with open('Predictions/pickle_Pred_Quran_Bukhari_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

In [0]:
class_names = ["0","1"]

# Generate the confusion matrix
cnf_matrix = confusion_matrix(y1_test, y1_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Quran Bukhari Confusion matrix')

plt.show()

In [0]:
import random
import math


# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
    
def attention2color(attention_score,predict,decision='INCORRECT'):
    r = 255 - int(attention_score * 255)
    if decision=='INCORRECT':
      color = rgb_to_hex((255, r, r))
    elif predict=='Quran':
      color = rgb_to_hex((r, 255, r))
    elif predict=='Bukhari':
      color = rgb_to_hex((r, r, 255))
    #print(color)
    return str(color)



y1_pred=[]

actual1=""
predict1=""
decision1=""

for i in range(len(X1_test)):

  tokenized_sample = str(x1_test_random[i]).split(" ")

  # Make predictions
  pred, attentions = model_with_attentions.predict(np.reshape(X1_test[i],(1,300)))
  y1_pred.append((int)(pred>0.5))

  if y1_test[i]==0:
    actual1="Quran"
  elif y1_test[i]==1:
    actual1="Bukhari"
  if y1_pred[i]==0:
    predict1="Quran"
  if y1_pred[i]==1:
    predict1="Bukhari"
  if y1_test[i]==y1_pred[i]:
    decision1="Correct"
  else:
    decision1="INCORRECT"


  token_attention_dic = {}
  max_score = 0.0
  min_score = 0.0

  for token, attention_score in zip(tokenized_sample, attentions[0][:len(tokenized_sample)]):
 
    token_attention_dic[token] = (math.sqrt(attention_score))

      
  # Build HTML String to viualize attentions
  html_text = "<hr><p style='font-size: large'><p><b>Actual: "+actual1+" Verdict: "+decision1+"</b></p><b>Text:  </b>"
  for token, attention in token_attention_dic.items():
      html_text += "<span style='background-color:{};'>{} <span> ".format(attention2color(attention, predict1, decision1),
                                                                          token)
  html_text += "</p>"
  # Display text enriched with attention scores 
  display(HTML(html_text))

# **Spooky Author All Dataset**

In [0]:
print("Fitting Model...")

pickle_load = open('Spooky Author/pickle_train_Spooky_Author_All_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_test_Spooky_Author_All_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_cleanXy_Spooky_Author_All_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-3916,:],y1_random[:-3916,:],x1_random[-3916:,:],y1_random[-3916:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)


y1_pred, attentions = model_with_attentions.predict(X1_test)


In [0]:
import pickle
with open('Predictions/pickle_Pred_Spooky_Author_All_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

In [0]:
import random
import math


# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
    
def attention2color(attention_score,predict,decision='INCORRECT'):
    r = 255 - int(attention_score * 255)
    if decision=='INCORRECT':
      color = rgb_to_hex((255, r, r))
    elif predict=='EAP':
      color = rgb_to_hex((r, 255, r))
    elif predict=='MWS':
      color = rgb_to_hex((r, r, 255))
    #print(color)
    return str(color)



y1_pred=[]

actual1=""
predict1=""
decision1=""

for i in range(len(X1_test)):
  
  tokenized_sample = str(x1_test_random[i]).split(" ")

  # Make predictions
  pred, attentions = model_with_attentions.predict(np.reshape(X1_test[i],(1,300)))
  
  y1_pred.append((int)(pred>0.5))

  if y1_test[i]==0:
    actual1="EAP"
  elif y1_test[i]==1:
    actual1="MWS"
  if y1_pred[i]==0:
    predict1="EAP"
  if y1_pred[i]==1:
    predict1="MWS"
  if y1_test[i]==y1_pred[i]:
    decision1="Correct"
  else:
    decision1="INCORRECT"



  token_attention_dic = {}
  max_score = 0.0
  min_score = 0.0
  
  for token, attention_score in zip(tokenized_sample, attentions[0][:len(tokenized_sample)]):
    
    token_attention_dic[token] = (math.sqrt(attention_score))

      
  # Build HTML String to viualize attentions
  html_text = "<hr><p style='font-size: large'><p><b>Actual: "+actual1+" Predict: "+predict1+" Verdict: "+decision1+"</b></p><b>Text:  </b>"
  for token, attention in token_attention_dic.items():
      
      html_text += "<span style='background-color:{};'>{} <span> ".format(attention2color(attention, predict1, decision1),
                                                                          token)
  html_text += "</p>"
  # Display text enriched with attention scores 
  display(HTML(html_text))


# **Spooky Author Binary**

In [0]:

print("Fitting Model...")

pickle_load = open('Spooky Author/pickle_train_Spooky_Author_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_test_Spooky_Author_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_cleanXy_Spooky_Author_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-2789,:],y1_random[:-2789,:],x1_random[-2789:,:],y1_random[-2789:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)


print("Evaluating Model...")
##########################################
y1_pred, attentions = model_with_attentions.predict(X1_test)
y_pred_class=[]

for y in y1_pred:
  y[0]=int(y[0]>0.5)

acc=accuracy_score(y1_test,y1_pred)

print("Eval with Spooky Author: "+str(Fold)+" " +str(acc))


In [0]:
import pickle
with open('Predictions/pickle_Pred_Spooky_Author_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

In [0]:
class_names = ["0","1"]

# Generate the confusion matrix
cnf_matrix = confusion_matrix(y1_test, y1_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Spooky Author Confusion matrix')

plt.show()

In [0]:
import random
import math


# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
    
def attention2color(attention_score,predict,decision='INCORRECT'):
    r = 255 - int(attention_score * 255)
    if decision=='INCORRECT':
      color = rgb_to_hex((255, r, r))
    elif predict=='EAP':
      color = rgb_to_hex((r, 255, r))
    elif predict=='MWS':
      color = rgb_to_hex((r, r, 255))
    #print(color)
    return str(color)


y1_pred=[]

actual1=""
predict1=""
decision1=""

for i in range(len(X1_test)):

  tokenized_sample = str(x1_test_random[i]).split(" ")


  # Make predictions
  pred, attentions = model_with_attentions.predict(np.reshape(X1_test[i],(1,300)))
  y1_pred.append((int)(pred>0.5))

  if y1_test[i]==0:
    actual1="EAP"
  elif y1_test[i]==1:
    actual1="MWS"
  if y1_pred[i]==0:
    predict1="EAP"
  if y1_pred[i]==1:
    predict1="MWS"
  if y1_test[i]==y1_pred[i]:
    decision1="Correct"
  else:
    decision1="INCORRECT"


  token_attention_dic = {}
  max_score = 0.0
  min_score = 0.0
  for token, attention_score in zip(tokenized_sample, attentions[0][:len(tokenized_sample)]):
    token_attention_dic[token] = (math.sqrt(attention_score))

      
  # Build HTML String to viualize attentions
  html_text = "<hr><p style='font-size: large'><p><b>Actual: "+actual1+" Predict: "+predict1+" Verdict: "+decision1+"</b></p><b>Text:  </b>"
  for token, attention in token_attention_dic.items():
      
      html_text += "<span style='background-color:{};'>{} <span> ".format(attention2color(attention, predict1, decision1),
                                                                          token)
  html_text += "</p>"
  # Display text enriched with attention scores 
  display(HTML(html_text))


# **Washington Irving**

In [0]:
print("Fitting Model...")

pickle_load = open('Rip Sketch/pickle_train_New_Sketch_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Rip Sketch/pickle_test_New_Sketch_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Rip Sketch/pickle_cleanXy_New_Sketch_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-1460,:],y1_random[:-1460,:],x1_random[-1460:,:],y1_random[-1460:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)


# evaluate the model
print("Evaluating Model...")
##########################################
y1_pred, attentions = model_with_attentions.predict(X1_test)
y_pred_class=[]

for y in y1_pred:
  y[0]=int(y[0]>0.5)

acc=accuracy_score(y1_test,y1_pred)

print("Eval with Spooky Author: "+str(Fold)+" " +str(acc))


In [0]:
import pickle
with open('Predictions/pickle_Pred_New_Sketch_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

In [0]:
class_names = ["Geoffrey Crayon", "Diedrich Knickerbocker"]

# Generate the confusion matrix
cnf_matrix = confusion_matrix(y1_test, y1_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Washington Irving Confusion matrix')


plt.show()

In [0]:
import random
import math

# VISUALIZATION
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
    
def attention2color(attention_score,predict,decision='INCORRECT'):
    r = 255 - int(attention_score * 255)
    if decision=='INCORRECT':
      color = rgb_to_hex((255, r, r))
    elif predict=='Geoffrey Crayon':
      color = rgb_to_hex((r, 255, r))
    elif predict=='Diedrich Knickerbocker':
      color = rgb_to_hex((r, r, 255))
    #print(color)
    return str(color)




y1_pred=[]

actual1=""
predict1=""
decision1=""

for i in range(len(X1_test)):

  tokenized_sample = str(x1_test_random[i]).split(" ")

  # Make predictions
  pred, attentions = model_with_attentions.predict(np.reshape(X1_test[i],(1,300)))

  y1_pred.append((int)(pred>0.5))

  if y1_test[i]==0:
    actual1="Geoffrey Crayon"
  elif y1_test[i]==1:
    actual1="Diedrich Knickerbocker"
  if y1_pred[i]==0:
    predict1="Geoffrey Crayon"
  if y1_pred[i]==1:
    predict1="Diedrich Knickerbocker"
  if y1_test[i]==y1_pred[i]:
    decision1="Correct"
  else:
    decision1="INCORRECT"


  token_attention_dic = {}
  max_score = 0.0
  min_score = 0.0

  for token, attention_score in zip(tokenized_sample, attentions[0][:len(tokenized_sample)]):

    token_attention_dic[token] = (math.sqrt(attention_score))

      
  # Build HTML String to viualize attentions
  html_text = "<hr><p style='font-size: large'><p><b>Actual: "+actual1+" Predict: "+predict1+" Verdict: "+decision1+"</b></p><b>Text:  </b>"
  for token, attention in token_attention_dic.items():

      html_text += "<span style='background-color:{};'>{} <span> ".format(attention2color(attention, predict1, decision1),
                                                                          token)
  html_text += "</p>"
  # Display text enriched with attention scores 
  display(HTML(html_text))


In [0]:
import random
import math


y1_pred=[]

att_Geoffrey_list=[]
att_Knickerbocker_list=[]

actual1=""
predict1=""
decision1=""

for i in range(len(X1_test)):

  pred, attentions = model_with_attentions.predict(np.reshape(X1_test[i],(1,300)))
  y1_pred.append((int)(pred>0.5))

  if y1_test[i]==0:
    actual1="Geoffrey"
  elif y1_test[i]==1:
    actual1="Knickerbocker"
  if y1_pred[i]==0:
    predict1="Geoffrey"
  if y1_pred[i]==1:
    predict1="Knickerbocker"
  if y1_test[i]==y1_pred[i]:
    decision1="Correct"
  else:
    decision1="INCORRECT"
  if y1_test[i]==0:
      att_Geoffrey_list.append(attentions)
  elif y1_test[i]==1:
      att_Knickerbocker_list.append(attentions)



In [0]:
import seaborn as sns
import matplotlib.pyplot as plt


att_Geoffrey_arr=np.sum(att_Geoffrey_list, axis=0,keepdims=False)
print(att_Geoffrey_arr.shape)
att_Geoffrey_arr=att_Geoffrey_arr.reshape((att_Geoffrey_arr.shape[1],))
att_Geoffrey_arr=att_Geoffrey_arr/(np.max(att_Geoffrey_arr))

plt.plot(att_Geoffrey_arr,c='y')
#plt.show()

att_Knickerbocker_arr=np.sum(att_Knickerbocker_list, axis=0,keepdims=False)
#print(att_Knickerbocker_arr)
att_Knickerbocker_arr=att_Knickerbocker_arr.reshape((att_Knickerbocker_arr.shape[1],))

att_Knickerbocker_arr=att_Knickerbocker_arr/(np.max(att_Knickerbocker_arr))
plt.plot(att_Knickerbocker_arr)


plt.show()



In [0]:
import pickle
with open('Attention_Score/pickle_Att_New_Sketch_1.pickle','wb') as f:
    pickle.dump((att_Geoffrey_arr,att_Knickerbocker_arr),f)

# **Random Reuter C50 All**

In [0]:
print("Fitting Model...")

pickle_load = open('Reuter_C50/pickle_train_Reuter_C50_All_Random_train_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_test_Reuter_C50_All_Random_test_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_cleanXy_Reuter_C50_All_Random_train_1.pickle', 'rb')
x1_train_random,y1_train_random,=pickle.load(pickle_load)

pickle_load = open('Reuter_C50/pickle_cleanXy_Reuter_C50_All_Random_test_1.pickle', 'rb')
x1_test_random,y1_test_random=pickle.load(pickle_load)


history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=20, batch_size=128,shuffle=True)


y1_pred, attentions = model_with_attentions.predict(X1_test)


In [0]:
import pickle
with open('Predictions/pickle_Pred_Random_Reuter_C50_All_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

# **Random Spooky Author Binary**

In [0]:
print("Fitting Model...")

pickle_load = open('Spooky Author/pickle_train_Spooky_Author_Random_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('/Spooky Author/pickle_test_Spooky_Author_Random_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_cleanXy_Spooky_Author_Random_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-2789,:],y1_random[:-2789,:],x1_random[-2789:,:],y1_random[-2789:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)


# evaluate the model
print("Evaluating Model...")
##########################################
y1_pred, attentions = model_with_attentions.predict(X1_test)
y_pred_class=[]

for y in y1_pred:
  y[0]=int(y[0]>0.5)

acc=accuracy_score(y1_test,y1_pred)

print("Eval with Spooky Author: "+str(Fold)+" " +str(acc))


In [0]:
import pickle
with open('Predictions/pickle_Pred_Spooky_Author_Random_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)

In [0]:
class_names = ["0","1"]

# Generate the confusion matrix
cnf_matrix = confusion_matrix(y1_test, y1_pred)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Spooky Author Random Confusion matrix')


plt.show()

# **Random Spooky Author All**

In [0]:
print("Fitting Model...")

pickle_load = open('Spooky Author/pickle_train_Spooky_Author_All_Random_1_100dim.pickle', 'rb')
X1_train,y1_train,embedding_matrix=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_test_Spooky_Author_All_Random_1_100dim.pickle', 'rb')
X1_test,y1_test,_=pickle.load(pickle_load)

pickle_load = open('Spooky Author/pickle_cleanXy_Spooky_Author_All_Random_1.pickle', 'rb')
x1_random,y1_random=pickle.load(pickle_load)

x1_train_random,y1_train_random,x1_test_random,y1_test_random=x1_random[:-3916,:],y1_random[:-3916,:],x1_random[-3916:,:],y1_random[-3916:,:]

history=model.fit(X1_train, y1_train, validation_split=0.1, epochs=15, batch_size=128,shuffle=True)



y1_pred, attentions = model_with_attentions.predict(X1_test)


In [0]:
import pickle
with open('Predictions/pickle_Pred_Spooky_Author_All_Random_1.pickle','wb') as f:
    pickle.dump((y1_test,y1_pred),f)